In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np
from scipy import stats


In [3]:
dataframe = pd.read_csv("train.csv", header=None)
dataset = dataframe.values
preCleanX = dataset[1:,1:-1].astype(float)
preCleanY = dataset[1:,-1]
for d in range(11):
  print(preCleanX[:,d].mean())
m = preCleanX.mean(axis=0)
s = preCleanX.std(axis=0)
print(m)
print(s)
preCleanX = (preCleanX - m)/s

notOutlierIndexes = (np.abs(stats.zscore(preCleanX)) < 3).all(axis=1)

X = preCleanX[notOutlierIndexes]
Y = preCleanY[notOutlierIndexes]



print(X.shape)
print(Y.shape)
for d in range(11):
  print(X[:,d].mean())


8.365175097276264
0.5276006809338522
0.26505836575875485
2.3988813229571986
0.08185603112840467
16.955982490272373
49.23686770428016
0.9967477023346303
3.31056906614786
0.6413083657587549
10.4149724383917
[ 8.3651751   0.52760068  0.26505837  2.39888132  0.08185603 16.95598249
 49.2368677   0.9967477   3.31056907  0.64130837 10.41497244]
[1.70497477e+00 1.73122029e-01 1.88221495e-01 8.58614628e-01
 2.37229406e-02 1.00072752e+01 3.29531241e+01 1.82702374e-03
 1.42286830e-01 1.37908185e-01 1.02857518e+00]
(1887, 11)
(1887,)
-0.030424825787348456
0.0027075781877047033
-0.03008423294357652
-0.11550326266213323
-0.07792507577590503
0.004901629544006037
-0.0009322448576926227
-0.033385112047886176
0.014585629398535452
-0.0808464213153493
-0.013131044430345606


In [35]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [4]:
def baseline_model():
 # create model
 model = Sequential()
 model.add(Dense(20, input_dim=X.shape[1], activation='relu'))
 model.add(Dense(10, activation='relu'))
 model.add(Dense(1))
 # Compile model
 model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
#  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [11]:
# estimator = KerasClassifier(build_fn=baseline_model, epochs=6, batch_size=1, verbose=2)
model = baseline_model()
model.fit(X, Y.astype(float), batch_size=1, epochs=5)

Epoch 1/5
1887/1887 [==============================] - 6s 3ms/step - loss: 3.5483 - mean_squared_error: 3.5483
Epoch 2/5
1887/1887 [==============================] - 5s 3ms/step - loss: 0.8215 - mean_squared_error: 0.8215
Epoch 3/5
1887/1887 [==============================] - 5s 3ms/step - loss: 0.6578 - mean_squared_error: 0.6578
Epoch 4/5
1887/1887 [==============================] - 5s 3ms/step - loss: 0.5790 - mean_squared_error: 0.5790
Epoch 5/5
1887/1887 [==============================] - 7s 4ms/step - loss: 0.5483 - mean_squared_error: 0.5483


In [12]:
sum(abs(model.predict(X[0:100].reshape((100,11))) - Y[0:100].astype(float).reshape(100,1)) > 0.5)

4/4 [==============================] - 0s 3ms/step


array([39])

In [13]:
dataframetest = pd.read_csv("test.csv", header=None)
testdataset = dataframetest.values
testX = testdataset[1:,1:].astype(float)
print(m)
print(s)
testX = (testX - m)/s
print(testX.shape)
predictedY = np.rint(model.predict(testX.reshape((1372,11)))).reshape(1372,)

testOutputData = {"Id": testdataset[1:,0], "quality": predictedY}
testOutputFrame = pd.DataFrame(testOutputData)
testOutputFrame.to_csv('best_submission.csv')
from google.colab import files
files.download('best_submission.csv')


[ 8.3651751   0.52760068  0.26505837  2.39888132  0.08185603 16.95598249
 49.2368677   0.9967477   3.31056907  0.64130837 10.41497244]
[1.70497477e+00 1.73122029e-01 1.88221495e-01 8.58614628e-01
 2.37229406e-02 1.00072752e+01 3.29531241e+01 1.82702374e-03
 1.42286830e-01 1.37908185e-01 1.02857518e+00]
(1372, 11)
43/43 [==============================] - 0s 1ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:

print(predictedY.shape)

43/43 [==============================] - 0s 1ms/step
(1372,)


In [50]:
testX = testdataset[1:,1:].astype(float)
mT = testX.mean(axis=0)
sT = testX.std(axis=0)
print(mT)
print(m)
print(sT)
print(s)


[ 8.36960641  0.52431122  0.27056851  2.41592566  0.08130685 17.16253644
 49.08928571  0.99666993  3.31222303  0.64726676 10.47308066]
[ 8.3651751   0.52760068  0.26505837  2.39888132  0.08185603 16.95598249
 49.2368677   0.9967477   3.31056907  0.64130837 10.41497244]
[1.66408700e+00 1.76877319e-01 1.85663566e-01 9.20819101e-01
 2.65563351e-02 9.97044665e+00 3.27917641e+01 1.82097833e-03
 1.41353102e-01 1.34743170e-01 1.02679587e+00]
[1.70497477e+00 1.73122029e-01 1.88221495e-01 8.58614628e-01
 2.37229406e-02 1.00072752e+01 3.29531241e+01 1.82702374e-03
 1.42286830e-01 1.37908185e-01 1.02857518e+00]


In [52]:
!pip3 install keras-visualizer
from keras_visualizer import visualizer 
visualizer(model, format='png', view=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
sum((np.abs(stats.zscore(testX)) >= 3).all(axis=1))

0

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, 'wine.js')

In [ ]:
from google.colab import files
!zip wine.zip wine.js/*
files.download('wine.zip')

updating: wine.js/group1-shard1of1.bin (deflated 5%)
updating: wine.js/model.json (deflated 73%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/5
1698/1698 - 4s - loss: 1.1504 - accuracy: 0.5306 - 4s/epoch - 3ms/step
Epoch 2/5
1698/1698 - 3s - loss: 1.0216 - accuracy: 0.5760 - 3s/epoch - 2ms/step
Epoch 3/5
1698/1698 - 4s - loss: 1.0002 - accuracy: 0.5760 - 4s/epoch - 2ms/step
Epoch 4/5
1698/1698 - 3s - loss: 0.9884 - accuracy: 0.5866 - 3s/epoch - 2ms/step
Epoch 5/5
1698/1698 - 3s - loss: 0.9808 - accuracy: 0.5978 - 3s/epoch - 2ms/step
189/189 - 0s - loss: 1.0986 - accuracy: 0.6296 - 400ms/epoch - 2ms/step
Epoch 1/5
1698/1698 - 5s - loss: 1.1758 - accuracy: 0.5359 - 5s/epoch - 3ms/step
Epoch 2/5
1698/1698 - 3s - loss: 1.0390 - accuracy: 0.5724 - 3s/epoch - 2ms/step
Epoch 3/5
1698/1698 - 3s - loss: 1.0188 - accuracy: 0.5777 - 3s/epoch - 2ms/step
Epoch 4/5
1698/1698 - 4s - loss: 1.0074 - accuracy: 0.5883 - 4s/epoch - 2ms/step
Epoch 5/5
1698/1698 - 3s - loss: 0.9964 - accuracy: 0.5978 - 3s/epoch - 2ms/step
189/189 - 0s - loss: 1.0263 - accuracy: 0.6138 - 400ms/epoch - 2ms/step
Epoch 1/5
1698/1698 - 5s - loss: 1.2048 - accu